In [1]:
import numpy as np
import xarray as xr

import CAFE_tools2 as jcafe

In [2]:
#Find variables for ROM from each ensemble member
nino34 = np.zeros((97,720))
nino3 = np.zeros((97,720))
wholeP = np.zeros((97,720))
westP = np.zeros((97,720))

for e in range(1,97):
    sst_full = xr.load_dataset('~/Large Datasets/anomaly_files/'+
                               jcafe.anomaly_file_name(1965,2018, 'sst',e_member=e))
    i20_full = xr.load_dataset('~/Large Datasets/anomaly_files/'+
                               jcafe.anomaly_file_name(1965,2018, 'i20',e_member=e))

    nino34[e] = jcafe.average_region(sst_full, jcafe.nino34_region).sst
    nino3[e]  = jcafe.average_region(sst_full, jcafe.nino3_region).sst
    wholeP[e] = jcafe.average_region(i20_full, jcafe.full_pacific).i20
    westP[e]  = jcafe.average_region(i20_full, jcafe.west_pacific).i20

CPU times: user 6.85 s, sys: 4.78 s, total: 11.6 s
Wall time: 14.3 s


In [8]:
#Fill in the 0th entry with the mean of all ensemble members
nino34[0] = np.mean(nino34[1:],0)
nino3[0] = np.mean(nino3[1:],0)
wholeP[0] = np.mean(wholeP[1:],0)
westP[0] = np.mean(westP[1:],0)

In [9]:
indices = xr.Dataset(data_vars={'nino34_sst':(('ensemble_member','time'),nino34),
                                'nino3_sst' :(('ensemble_member','time'),nino3),
                                'wholeP_i20':(('ensemble_member','time'),wholeP),
                                'westP_i20' :(('ensemble_member','time'),westP)},
                     coords = {'time':sst_full.time,
                               'ensemble_member':np.arange(97,dtype=np.int64)})

In [10]:
indices.to_netcdf('saved_indices.nc')